In [ ]:
import datetime as dt
from typing import NamedTuple

from micro_namedtuple_sqlite_persister.persister import Engine, enable_included_adaptconverters
enable_included_adaptconverters()

In [ ]:
class MyModel(NamedTuple):
    id: int | None
    name: str
    date: dt.datetime
    score: float

In [ ]:
# engine = Engine(":memory:")
engine = Engine("example.db")
engine.ensure_table_created(MyModel)

In [ ]:
row = MyModel(None, "Drew", dt.datetime.now(), 6.5)
row = engine.insert(row)
engine.connection.commit()
row

In [ ]:
engine.get(MyModel,row.id)

In [ ]:
from random import random
randfloat = random()
engine.update(row._replace(score=row.score*randfloat))
engine.connection.commit()

In [ ]:
row2 = engine.insert(MyModel(None, "foo", dt.datetime.now(), 6.5))
print(row2)

print(f"\nAbout to delete the recently insert row.id: {row2.id}")
engine.delete(MyModel, row2.id)

engine.connection.commit()

In [ ]:
# insert many
for i in range(25000):
    engine.insert(MyModel(None, "foo", dt.datetime.now(), random()*100))

engine.connection.commit()

In [ ]:
# update many
for id in range(1, 5001):
    engine.update(MyModel(id, "drew", dt.datetime.now(), random()*100))

engine.connection.commit()

# Querying

In [ ]:
from micro_namedtuple_sqlite_persister.query import select, gt,and_, eq

M, q = select(MyModel, where=gt(MyModel.score, 99.7))

c = 1
for r in engine.query(M,q):
    print(f"{r.score:5.1f}", end=" ")
    if c% 50 == 0:
        print()
    c += 1
print()



In [ ]:
# Demo raw sql
class AverageScoreResults(NamedTuple):
    avg_score: float
    scorecount: int

sql = 'select avg(score),count(*) from MyModel'

result = next(iter(engine.query(AverageScoreResults, sql)))
print(f'The table has {result.scorecount} rows, with and average of {result.avg_score:0.2f}')

# Persisting Custom Types: Adapt/Convert

In [ ]:
import pandas as pd
import pickle

from micro_namedtuple_sqlite_persister.persister import register_adapt_convert

def adapt_df(obj: pd.DataFrame) -> bytes:
    return pickle.dumps(obj)


def convert_df(data: bytes) -> pd.DataFrame:
    return pickle.loads(data)


register_adapt_convert(pd.DataFrame, adapt_df, convert_df, overwrite=True)


class MyModel2(NamedTuple):
    id: int | None
    name: str
    df: pd.DataFrame

engine.ensure_table_created(MyModel2)

df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})
row = engine.insert(MyModel2(None, "foo", df))

engine.get(MyModel2, row.id).df


# Error Cases

In [ ]:
# inserting a row with an id that already exists will raise an error
engine.insert(MyModel(row.id, "bar", dt.datetime.now(), 3.14))

In [ ]:
# Trying to update a row without specifying an id will raise an error
engine.update(MyModel(None, "bar", dt.datetime.now(), 3.14))

In [ ]:
# Raises an error if the id does not exist
engine.update(MyModel(878787879879, "bar", dt.datetime.now(), 3.14))